In [1]:
import sys
sys.path.append('../')
import env

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
READ_FILE = 'Origin'

In [4]:
def main(args, array):
    df = pd.read_csv(env.get_full_path(READ_FILE, args),
                     names=env.get_col_names(),
                     encoding='Shift_JISx0213')
    array[args.dir][args.ratio][args.seed][args.csv] = df.copy()

csv_array = {}
env.for_default_init(main, csv_array, csv=['od'])

In [5]:
people='p10000'
ratio='r4'
seed='s123'
csv='od'
df = csv_array[people][ratio][seed][csv].copy()

In [6]:
df_hoge = df.loc[0:3].copy()
df_hoge

,c00,c01,c02,c03,c04,c05,c06,c07,c08,c09,...,c20,c21,c22,c23,c24,c25,c26,c27,c28,c29
0,15131,Vehicle,TRUE,国道157号m(census)@342,国道157号l(census)@361,国道157号k(census)@383,国道157号j(census)@408,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12929,Vehicle,TRUE,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19899,Vehicle,TRUE,石川県道17号i(census)@505,石川県道200号a(census)@552,,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13348,Vehicle,TRUE,石川県道144号b(census)@673,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_hoge.applymap(lambda x: (x.split('(census)')[0]) if (type(x) is str) and ('(census)' in x) else x)

,c00,c01,c02,c03,c04,c05,c06,c07,c08,c09,...,c20,c21,c22,c23,c24,c25,c26,c27,c28,c29
0,15131,Vehicle,TRUE,国道157号m,国道157号l,国道157号k,国道157号j,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12929,Vehicle,TRUE,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19899,Vehicle,TRUE,石川県道17号i,石川県道200号a,,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13348,Vehicle,TRUE,石川県道144号b,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
import re
pattern = '(.*)\(census\)@(\d+)'

series = df_hoge.loc[0].copy()
row = series.values.tolist()
for i in range(len(row)):
    if (i >= 3) and (type(row[i]) is str) and ('(census)' in row[i]):
                result = re.compile(pattern).match(row[i])
                print(result[1], result[2])
#         road_name = row[i].split('(census)')[0]
#         repatter = re.compile(pattern)
#         result = repatter.match(row[i])

 国道157号m 342
 国道157号l 361
 国道157号k 383
 国道157号j 408


In [9]:
def func_road_to_area(args):
    df = pd.read_csv(env.get_full_path('include_area_-1', args),
                     encoding='Shift_JISx0213')
    df = df.loc[:, ['road', 'area']]
    for row in np.asanyarray(df):
        road_to_area[row[0]] = float(row[1])

    road_to_area[np.nan] = np.nan
    
road_to_area = {}
env.for_default(func_road_to_area, csv=['census'])

In [10]:
import re
pattern = '(.*)\(census\)@(\d+)'
def extract_only_area(series):
    row = series.values.tolist()
    df_new = row[0:3]
    df_base = []

    for i in range(len(row)):
        if (i >= 3) and (type(row[i]) is str) and ('(census)' in row[i]):
            road_name = row[i].split('(census)')[0]
            _area = road_to_area[road_name]
            df_base.append(_area)
            
            result = re.compile(pattern).match(row[i])
            road_name1 = result[1]
            time = result[2]
            print(road_name1, time)

    df_new.extend(df_base)

    return pd.Series(df_new)

In [11]:
df_area = df_hoge.apply(lambda x: extract_only_area(x), axis=1)

 国道157号m 342
 国道157号l 361
 国道157号k 383
 国道157号j 408
 国道157号m 342
 国道157号l 361
 国道157号k 383
 国道157号j 408
 石川県道17号i 505
 石川県道200号a 552
 石川県道144号b 673


In [12]:
df_area

,0,1,2,3,4,5,6
0,15131,Vehicle,TRUE,50.0,40.0,40.0,40.0
1,12929,Vehicle,TRUE,NaN,NaN,NaN,NaN
2,19899,Vehicle,TRUE,49.0,49.0,NaN,NaN
3,13348,Vehicle,TRUE,40.0,NaN,NaN,NaN
